In [2]:
import tkinter as tk

# ---------------- Flashcards (Python cheat-sheet style) ----------------
flashcards = [
    {"question": "What is a List in Python? Give a short definition and an example (e.g., [1, 2, 3]).",
     "answer": "An ordered, mutable collection. Example: my_list = [1, 2, 3]"},
    {"question": "What does len() do in Python? Provide a brief explanation.",
     "answer": "Returns the number of items in a collection (like list, tuple, string)."},
    {"question": "What is a Tuple and how is it different from a List?",
     "answer": "A tuple is ordered and immutable. Lists are ordered and mutable."},
    {"question": "What is a Dictionary in Python?",
     "answer": "A collection of key:value pairs, unordered (in older versions) and accessed by key."}
]

index = 0

# --------------------- GUI / Canvas Setup ---------------------
window = tk.Tk()
window.title("Python Flashcard Cheat Sheet")
window.geometry("520x520")
window.config(bg="#e8f0fe")

TITLE_Y = 30
CARD_CENTER_X = 260
CARD_CENTER_Y = 240
CARD_HALF_W = 180
CARD_HALF_H = 120
ANIM_STEPS = 12

title_label = tk.Label(window, text="🐍 PYTHON FLASHCARD", font=("Consolas", 20, "bold"), bg="#e8f0fe")
title_label.place(x=130, y=8)

card_canvas = tk.Canvas(window, width=480, height=360, bg="#e8f0fe", highlightthickness=0)
card_canvas.place(x=20, y=60)

# Draw the card rectangle with shadow
shadow = card_canvas.create_rectangle(
    CARD_CENTER_X - CARD_HALF_W + 6,
    CARD_CENTER_Y - CARD_HALF_H + 6,
    CARD_CENTER_X + CARD_HALF_W + 6,
    CARD_CENTER_Y + CARD_HALF_H + 6,
    fill="#cfd8ff",
    outline="",
    tags="card"
)

card_rect = card_canvas.create_rectangle(
    CARD_CENTER_X - CARD_HALF_W,
    CARD_CENTER_Y - CARD_HALF_H,
    CARD_CENTER_X + CARD_HALF_W,
    CARD_CENTER_Y + CARD_HALF_H,
    fill="white",
    outline="#9fb3ff",
    width=3,
    tags="card"
)

# Header text on the card
card_canvas.create_text(CARD_CENTER_X, CARD_CENTER_Y - CARD_HALF_H + 28,
                        text="Python Cheat-Sheet", font=("Consolas", 14, "bold"), fill="#1f4ea6", tags="card")

# Placeholder for question/answer text (tagged 'body_text')
body_text = card_canvas.create_text(
    CARD_CENTER_X,
    CARD_CENTER_Y,
    text="",
    font=("Consolas", 16),
    fill="#000000",
    width=CARD_HALF_W * 2 - 20,  # wrapping width (keeps text inside card)
    justify="center",
    tags="body_text"
)

# --------------------- Text Display Functions ---------------------
def show_question_text():
    """Display the question side; enable flip, disable next."""
    card_canvas.itemconfig(body_text, text=flashcards[index]["question"], fill="#111111")
    flip_btn.config(state="normal")
    next_btn.config(state="disabled")

def show_answer_text():
    """Display the answer side; enable next, disable flip."""
    card_canvas.itemconfig(body_text, text=flashcards[index]["answer"], fill="#065f46")
    flip_btn.config(state="disabled")
    next_btn.config(state="normal")

# --------------------- Animations (non-blocking using after) ---------------------
def animate_flip(step=0, shrinking=True):
    """
    Simulate flip by shrinking card width horizontally to a narrow line,
    switch text at midpoint, then expand back.
    """
    # compute shrink factor per step (we change rectangle coords)
    if shrinking:
        t = step / ANIM_STEPS  # 0 -> 1
        half_w = int(CARD_HALF_W * (1 - 0.95 * t))  # shrink toward center (not full 0)
    else:
        t = step / ANIM_STEPS
        half_w = int(CARD_HALF_W * (0.05 + 0.95 * t))  # expand back to full

    # update rectangle coords (both rect and shadow)
    left = CARD_CENTER_X - half_w
    right = CARD_CENTER_X + half_w
    card_canvas.coords(card_rect,
                       left, CARD_CENTER_Y - CARD_HALF_H,
                       right, CARD_CENTER_Y + CARD_HALF_H)
    card_canvas.coords(shadow,
                       left + 6, CARD_CENTER_Y - CARD_HALF_H + 6,
                       right + 6, CARD_CENTER_Y + CARD_HALF_H + 6)

    # keep header position fixed, update body text wrap width and position
    card_canvas.coords(body_text, CARD_CENTER_X, CARD_CENTER_Y)
    # set wrap width to slightly inside current width (avoid text overflow)
    card_canvas.itemconfig(body_text, width=max(40, half_w * 2 - 40))

    if shrinking:
        if step < ANIM_STEPS:
            window.after(18, lambda s=step + 1: animate_flip(s, shrinking=True))
        else:
            # midpoint reached - swap to answer
            show_answer_text()
            # expand back
            window.after(30, lambda: animate_flip(0, shrinking=False))
    else:
        if step < ANIM_STEPS:
            window.after(18, lambda s=step + 1: animate_flip(s, shrinking=False))
        else:
            # finished expanding - done
            pass

def animate_slide_out_in(direction=1, step=0):
    """
    Slide the card to the right (direction=1) or left (direction=-1) and bring new content in.
    Uses move() so coordinates stay consistent.
    """
    if step < ANIM_STEPS:
        dx = int(20 * direction)  # moving amount per step
        card_canvas.move("all", dx, 0)
        window.after(18, lambda: animate_slide_out_in(direction, step + 1))
    else:
        # we've moved out; update text for next card and slide back in
        show_question_text()
        # slide back in opposite direction
        animate_slide_back(-direction, 0)

def animate_slide_back(direction, step=0):
    if step < ANIM_STEPS:
        dx = int(20 * direction)
        card_canvas.move("all", dx, 0)
        window.after(18, lambda: animate_slide_back(direction, step + 1))
    else:
        # ensure card is exactly centered after animation
        # compute current coords and recenter if necessary
        coords = card_canvas.coords(card_rect)
        cur_center_x = (coords[0] + coords[2]) / 2
        delta = CARD_CENTER_X - cur_center_x
        if abs(delta) > 0:
            card_canvas.move("all", delta, 0)

# --------------------- Button Handlers ---------------------
def flip_card():
    """Triggered when user clicks flip/check answer."""
    flip_btn.config(state="disabled")  # prevent re-click during animation
    animate_flip(0, shrinking=True)

def next_card():
    """Go to the next card with a slide-out/slide-in animation."""
    global index
    index += 1
    if index >= len(flashcards):
        # finished
        card_canvas.itemconfig(body_text, text="🎉 Completed All Flashcards!", fill="#0b5")
        flip_btn.config(state="disabled")
        next_btn.config(state="disabled")
    else:
        # slide out to the right, update text in the middle of animation
        next_btn.config(state="disabled")
        animate_slide_out_in(direction=1, step=0)

# --------------------- UI Controls ---------------------
answer_entry = tk.Entry(window, font=("Arial", 14))
answer_entry.place(x=120, y=440, width=280)

# Note: The entry is not required to flip; this version keeps flipping separate.
# If you want to validate typed answers, we can wire the entry into a checker.

flip_btn = tk.Button(window, text="Flip / Show Answer", font=("Arial", 12), command=flip_card, bg="#3b82f6", fg="white")
flip_btn.place(x=120, y=480, width=140)

next_btn = tk.Button(window, text="Next →", font=("Arial", 12), command=next_card, bg="#10b981", fg="white", state="disabled")
next_btn.place(x=300, y=480, width=100)

# show first question
show_question_text()

window.mainloop()


TclError: no display name and no $DISPLAY environment variable